In [1]:
from transformers import RobertaTokenizerFast
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel
import transformers
from transformers import BertModel, BertTokenizer
from torch import nn
import torch

In [2]:
MAX_LEN = 128
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
tokenizer_folder = '/users/kent/jmaharja/drugAbuse/output/oct2022/TokRoBERTa'
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_folder, max_len=MAX_LEN)

In [4]:
class TweetModel(RobertaPreTrainedModel):
    def __init__(self, conf, n_classes):
        super(TweetModel, self).__init__(conf)
        self.roberta = transformers.RobertaModel.from_pretrained('/users/kent/jmaharja/drugAbuse/output/oct2022/RoBERTaMLM/', config=conf)
        self.drop_out = nn.Dropout(0.5)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.classifier = nn.Linear(768, n_classes)
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.roberta(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.drop_out(pooler)
        output = self.classifier(pooler)
        return output

In [5]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=8192,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
    hidden_size=768,
    pad_token_id=1
)

model = TweetModel(config, 2)
model = model.to(device)

Some weights of the model checkpoint at /users/kent/jmaharja/drugAbuse/output/oct2022/RoBERTaMLM/ were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /users/kent/jmaharja/drugAbuse/output/oct2022/RoBERTaMLM/ and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream

In [6]:

model.load_state_dict(torch.load('/users/kent/jmaharja/drugAbuse/finetune/checkpoint/best_ftc_model_state2023_02_20-03_15PM.bin'))
# model = torch.load('/users/kent/jmaharja/drugAbuse/finetune/checkpoint/best_ftc_model_state2023_02_20-03_15PM.bin')

<All keys matched successfully>

In [7]:
tweet = "Some shots from my shoot for F.I.T. Studio. Coach Fred"
encoded_tweet = tokenizer.encode_plus(
    tweet,
    max_length=MAX_LEN,
    truncation=True,
    pad_to_max_length=True,
    add_special_tokens=True,
    padding='max_length',
    return_token_type_ids=True,
    return_tensors='pt'
)

In [8]:
input_ids = encoded_tweet['input_ids'].to(device)
attention_mask = encoded_tweet['attention_mask'].to(device)
token_type_ids = encoded_tweet['token_type_ids'].to(device)

In [9]:
output = model(input_ids, attention_mask, token_type_ids)
_, prediction = torch.max(output, dim=1)
print(f'Tweet text: {tweet}')
print(f'Substance type  : {prediction}')

Tweet text: Some shots from my shoot for F.I.T. Studio. Coach Fred
Substance type  : tensor([0], device='cuda:0')


In [10]:
# outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

In [11]:
import json
import torch

In [14]:
%curl -X POST -H "Content-Type: application/json" -d '{"tweet":"Some shots from my shoot for F.I.T. Studio. Coach Fred"}' http://localhost:5000/


UsageError: Line magic function `%curl` not found.


In [18]:
tensor = torch.tensor([1, 2, 3])

# Serialize tensor to JSON string
tensor_json = json.dumps(prediction.tolist())

# Print JSON string
print(tensor_json)

[0]


In [20]:
tweet + "--->" + tensor_json


'Some shots from my shoot for F.I.T. Studio. Coach Fred--->[0]'